# Option settings 

In [1]:
n_epochs=100 #number of epochs of training
batch_size=1000 #size of the batches
lr=0.0002 #adam: learning rate
b1=0.5  #"adam: decay of first order momentum of gradient")
b2=0.999 #adam: decay of first order momentum of gradient")
n_cpu=8
latent_dim=100
img_size=250
channels=1
n_critic=5
clip_value=0.01
sample_interval=1000
img_shape = (channels, img_size, img_size)
crop_size = 400
print(img_shape)

(1, 250, 250)


# Static Code

In [2]:
from tensorboard_logger import configure, log_value
import tensorboard
import argparse
import os
import numpy as np
import math
import sys
import glob
import time
import random
#import utils import Logger

# torchvision stuff
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchvision import datasets


# torch stuff
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset







from PIL import Image


class ImageDataset(Dataset):
    def __init__(self, folder_path, transforms_=None):
        self.transform = transforms.Compose(transforms_)
        self.files = sorted(glob.glob('%s/*.*' % folder_path))

    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img = self.transform(img)
        return img

    def __len__(self):
        return len(self.files)
    
    
cuda = True if torch.cuda.is_available() else False
print(cuda)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity
    
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty


True


# Initialize Models and Load Settings

In [3]:
# Loss weight for gradient penalty
lambda_gp = 10


# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# If cuda, use cuda
if cuda:
    generator.cuda()
    discriminator.cuda()
    
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b2, b2))
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


# Load Previous model

In [4]:
state_g = torch.load("/home/jgmeyer2/vangan/gans/models/g250px_jpegs_bw.model")
state_d = torch.load("/home/jgmeyer2/vangan/gans/models/d250px_jpegs_bw.model")


generator.load_state_dict(state_g['state_dict'])
optimizer_G.load_state_dict(state_g['optimizer'])

discriminator.load_state_dict(state_d['state_dict'])
optimizer_D.load_state_dict(state_d['optimizer'])




# INitialize logger

In [5]:
configure("runs/bw250n2",flush_secs=5)

# Training loop

# make training function

In [6]:


def train(dataloader, batch_size, n_epochs):
    start = time.time()
    batches_done=0
    for epoch in range(n_epochs):
        batchtimes=[float()]
        for i, imgs in enumerate(dataloader):
            ## monitor time

            # Configure input
            real_imgs = Variable(imgs.type(Tensor))
            # ---------------------
            #  Train Discriminator
            # ---------------------
            optimizer_D.zero_grad()
            # Sample noise as generator input
            z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
            # Generate a batch of images
            fake_imgs = generator(z)
            # Real images
            real_validity = discriminator(real_imgs)
            # Fake images
            fake_validity = discriminator(fake_imgs)
            # Gradient penalty
            gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
            # Adversarial loss
            d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty
            d_loss.backward()
            optimizer_D.step()
            optimizer_G.zero_grad()
            # Train the generator every n_critic steps
            if i % n_critic == 0:
                # -----------------
                #  Train Generator
                # -----------------
                # Generate a batch of images
                fake_imgs = generator(z)
                # Loss measures generator's ability to fool the discriminator
                # Train on fake images
                fake_validity = discriminator(fake_imgs)
                g_loss = -torch.mean(fake_validity)
                g_loss.backward()
                optimizer_G.step()
                #logger.log(d_loss, g_loss, epoch, batches_done, num_batches)
                if batches_done % sample_interval/10 ==0:
                    log_value('g_loss', g_loss, batches_done)
                    log_value('d_loss', d_loss, batches_done)

                if batches_done % sample_interval/10 == 0:
                    save_image(fake_imgs.data[:25], "molpics250px_bw/%d_b.png" % batches_done, nrow=5, normalize=True)

                end = time.time()
                batchtime = (end - start)/5

                print(
                    "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [batch time: %f]"
                    % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item(), batchtime)
                )
                batchtimes.append(batchtime/batch_size)
                start = time.time()
                batches_done += n_critic
        print("average time per picture = " +str(np.mean(batchtimes)))
        print("minutes per 100,000 pictures = "+str((np.mean(batchtimes)*100000)/60))

# Input data 

In [ ]:
folder_path = "./zinc100k250pxJPEGbw/"
n_epochs=10000
batch_size = 1000
transforms_ = [ transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset(folder_path, transforms_=transforms_),
                        batch_size=batch_size, shuffle=True, num_workers=8)
train(dataloader, batch_size, n_epochs)

[Epoch 0/10000] [Batch 0/100] [D loss: 2.353823] [G loss: 272.677124] [batch time: 0.542465]
[Epoch 0/10000] [Batch 5/100] [D loss: 1.987182] [G loss: 289.494385] [batch time: 0.301251]
[Epoch 0/10000] [Batch 10/100] [D loss: 1.581028] [G loss: 305.757019] [batch time: 0.302904]
[Epoch 0/10000] [Batch 15/100] [D loss: 1.327439] [G loss: 321.162811] [batch time: 0.296978]
[Epoch 0/10000] [Batch 20/100] [D loss: 1.391119] [G loss: 335.550140] [batch time: 0.297094]
[Epoch 0/10000] [Batch 25/100] [D loss: 1.299007] [G loss: 348.966766] [batch time: 0.301552]
[Epoch 0/10000] [Batch 30/100] [D loss: 1.251305] [G loss: 361.027222] [batch time: 0.297408]
[Epoch 0/10000] [Batch 35/100] [D loss: 1.613397] [G loss: 371.269165] [batch time: 0.302569]
[Epoch 0/10000] [Batch 40/100] [D loss: 2.605494] [G loss: 379.260498] [batch time: 0.301731]
[Epoch 0/10000] [Batch 45/100] [D loss: 3.365986] [G loss: 384.848602] [batch time: 0.307398]
[Epoch 0/10000] [Batch 50/100] [D loss: 3.647167] [G loss: 387

[Epoch 4/10000] [Batch 15/100] [D loss: 8.032801] [G loss: 426.383636] [batch time: 0.300741]
[Epoch 4/10000] [Batch 20/100] [D loss: 7.607787] [G loss: 419.692322] [batch time: 0.302361]
[Epoch 4/10000] [Batch 25/100] [D loss: 6.726378] [G loss: 409.705505] [batch time: 0.309457]
[Epoch 4/10000] [Batch 30/100] [D loss: 5.724051] [G loss: 396.693024] [batch time: 0.309523]
[Epoch 4/10000] [Batch 35/100] [D loss: 4.527613] [G loss: 380.902954] [batch time: 0.314141]
[Epoch 4/10000] [Batch 40/100] [D loss: 3.598086] [G loss: 362.556519] [batch time: 0.315589]
[Epoch 4/10000] [Batch 45/100] [D loss: 2.609808] [G loss: 342.056122] [batch time: 0.316434]
[Epoch 4/10000] [Batch 50/100] [D loss: 1.685929] [G loss: 319.790771] [batch time: 0.302858]
[Epoch 4/10000] [Batch 55/100] [D loss: 1.091435] [G loss: 295.912781] [batch time: 0.303462]
[Epoch 4/10000] [Batch 60/100] [D loss: 0.639137] [G loss: 271.073212] [batch time: 0.298985]
[Epoch 4/10000] [Batch 65/100] [D loss: 0.446929] [G loss: 2

# Save models

In [24]:
os.makedirs("/home/jgmeyer2/vangan/gans/models",exist_ok=True)
PATH = "/home/jgmeyer2/vangan/gans/models/"
modelid="250px_jpegs_bw"
epoch=100

state_g = {
    'epoch': epoch,
    'state_dict': generator.state_dict(),
    'optimizer': optimizer_G.state_dict()
    }
torch.save(state_g, PATH+"g"+modelid+".model")

state_d = {
    'epoch': epoch,
    'state_dict': discriminator.state_dict(),
    'optimizer': optimizer_D.state_dict()
    }
torch.save(state_d, PATH+"d"+modelid+".model")
print("saved models @")
print(epoch)


#def save_model(net, optim, ckpt_fname):
#    state_dict = net.module.state_dict()
#    for key in state_dict.keys():
#        state_dict[key] = state_dict[key].cpu()
#        torch.save({
#            'epoch': epoch,                                                                                                                                                                                     
#            'state_dict': state_dict,                                                                                                                                                                                
#            'optimizer': optim},                                                                                                                                                                                     
#            ckpt_fname)

saved models @
100


# view training process

In [ ]:
#tensorboard
!python -m tensorboard.main --logdir runs